In [1]:
import pandas as pd
import joblib

In [2]:
df = pd.read_excel("data/tweet_bersih_100.xlsx") #data latih yang digunakan tweet_bersih_100.xlsx
twet = df.tweet.tolist()
label = df.label.tolist()
df.head() #menampilkan lima teraatas dari data bersih

,Unnamed: 0,tweet,label
0,0,buka visi misi begitu pukau sehingga buat pres...,positive
1,1,lebih baik pakai teknologi lama kaya negara in...,positive
2,2,kok arguing without menang kokarguing arguingw...,positive
3,3,jalan politik politik satu bukan politik pecah...,positive
4,4,juga fans lihat tampil serang hahaha jugafans ...,negative


In [3]:
pd.value_counts(df['label']).plot.bar(title='Data Sentimen')
print(pd.value_counts(df['label']))

positive    719
negative    681
Name: label, dtype: int64


In [4]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = twet
vectorizer = TfidfVectorizer()
save = vectorizer.fit(corpus)
joblib.dump(save,"model/tdidf.w")
X = vectorizer.fit_transform(corpus)
len(vectorizer.get_feature_names())#jumlah line fitur

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [6]:
k = 10
y = np.array(label)
kf = KFold(n_splits=k) #membuat model kfold sejumlah k
kf.get_n_splits(X)
# print(kf)
alpah_list = np.arange(0.1, 1+0.001, 0.001)
rerata = list()
for alpha in alpah_list:
    clf = MultinomialNB(alpha = alpha)
    sc_10 = list()
    for train_index, test_index in kf.split(X):
    #     print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf.fit(X_train, y_train)
        sc = clf.score(X_test, y_test)
        sc_10.append(sc)
#     print( sum(sc_10)/k, "|", alpha)
    rerata.append(sum(sc_10)/k)
dict_ = {
    "alpha":alpah_list,
    "score":rerata
}
df_hasil = pd.DataFrame.from_dict(dict_)
sorted_ = df_hasil.sort_values("score", axis = 0, ascending = False)
sorted_.head()

,alpha,score
0,0.100,0.799
7,0.107,0.798
10,0.110,0.798
9,0.109,0.798
8,0.108,0.798


In [7]:
sorted_.alpha.tolist()[0]

0.1

In [8]:
clf = MultinomialNB(alpha = sorted_.alpha.tolist()[0])
save = clf.fit(X, y) 
joblib.dump(save,"model/naive_bayes.m")

['model/naive_bayes.m']

In [9]:
save.score(X, y)

0.993

In [10]:
x = save.predict(X)

In [11]:
ixx = 0
labelA = list()
labelP= list()

for i, j in zip(label, x):
    if i !=j:
        print(ixx)
        print(j)
        print("label asli",i)
        print(twet[ixx])
        print()
    ixx+=1

541
negative
label asli positive
tidak paham ekonomi investasi asing lingkup bisnis di-framing olah negara asai asing cara militan tidakpaham pahamekonomi ekonomiinvestasi investasiasing asinglingkup lingkupbisnis bisnisdi-framing di-framingolah olahnegara negaraasai asaiasing asingcara caramilitan

716
negative
label asli positive
rendah dukung tahana rupa bukti rupa bukti bila cara tak langsung sedang hukum publik rendahdukung dukungtahana tahanarupa rupabukti buktirupa rupabukti buktibila bilacara caratak taklangsung langsungsedang sedanghukum hukumpublik

830
positive
label asli negative
pak all keluar debat malam semua umpan lawan lahap habis bahkan jawab kerap jadi serang balik pakall allkeluar keluardebat debatmalam malamsemua semuaumpan umpanlawan lawanlahap lahaphabis habisbahkan bahkanjawab jawabkerap kerapjadi jadiserang serangbalik

858
positive
label asli negative
apresiasi saudara di papua pakde jadi percuma kalian tebar bohong papua derita apresiasisaudara saudaradi dipa